In [1]:
from pathlib import Path
import pandas as pd
import sys 
import os

# Definir la ruta base
data_path = Path('..') / 'data' / 'raw'
src_path = Path('..') / 'src' 

sys.path.append(str(src_path))
from data_cleaning_utils import *

# Leer los archivos
# Probar con diferentes separadores y encoding
salario_edad = pd.read_csv(data_path / 'salario_medio bruto_edad.csv', 
                  sep=';',  # Muchos CSVs españoles usan punto y coma
                  encoding='utf-8')

salario_comunidad = pd.read_csv(data_path / 'salario_medio_bruto_comunidad.csv',
                  sep=';',
                  encoding='utf-8')

ipv = pd.read_csv(data_path / 'ipv historico.csv',
                  sep=';',
                  encoding='utf-8')

superficie_media = pd.read_csv(data_path / 'superficie util vivienda valor medio del periodo.csv',
                  sep=';',
                  encoding='utf-8')

interes_fijo = pd.read_csv(data_path / 'tipo interes medio inicio hipotecas constituidas fijo o variable.csv',
                  sep=';',
                  encoding='utf-8')
idealista_excel = pd.read_excel(data_path / 'Copia_idealista_metros_cuadrados.xlsx')


# EXPLORACIÓN INICIAL DE DATASETS PARA ANÁLISIS HIPOTECARIO

In [2]:
# Diccionario de datasets para iterar
datasets = {
    'Salarios por Edad': salario_edad,
    'Salarios por Comunidad': salario_comunidad, 
    'IPV Histórico': ipv,
    'Excel Idealista' : idealista_excel,
    'Superficie Media': superficie_media,
    'Tipos Interés': interes_fijo
}

In [3]:
for name, df in datasets.items():
    print(f"DATASET: {name}")
    print(f"Shape: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    print(f"Primeras 3 filas:")
    print(df.head(3))
    print(f"Tipos de datos:")
    print(df.dtypes)
    print(f"Valores nulos por columna:")
    print(df.isnull().sum())
    print("-" * 60)

DATASET: Salarios por Edad
Shape: (3564, 5)
Columnas: ['Tipo de jornada', 'Grupo de edad', 'Decil', 'Periodo', 'Total']
Primeras 3 filas:
  Tipo de jornada Grupo de edad        Decil  Periodo     Total
0           Total         Total  Total decil     2023  2.273,01
1           Total         Total  Total decil     2022  2.118,79
2           Total         Total  Total decil     2021  2.076,45
Tipos de datos:
Tipo de jornada    object
Grupo de edad      object
Decil              object
Periodo             int64
Total              object
dtype: object
Valores nulos por columna:
Tipo de jornada    0
Grupo de edad      0
Decil              0
Periodo            0
Total              0
dtype: int64
------------------------------------------------------------
DATASET: Salarios por Comunidad
Shape: (11880, 5)
Columnas: ['Tipo de jornada', 'Comunidades y Ciudades Autonómas', 'Decil', 'Periodo', 'Total']
Primeras 3 filas:
  Tipo de jornada Comunidades y Ciudades Autonómas        Decil  Periodo  \
0

# ANÁLISIS EXPLORATORIO ESPECÍFICO POR DATASET

In [4]:
salario_comunidad['Comunidades y Ciudades Autonómas'].value_counts()

Comunidades y Ciudades Autonómas
Total Nacional                    594
01 Andalucía                      594
18 Ceuta                          594
17 Rioja, La                      594
16 País Vasco                     594
15 Navarra, Comunidad Foral de    594
14 Murcia, Región de              594
13 Madrid, Comunidad de           594
12 Galicia                        594
11 Extremadura                    594
10 Comunitat Valenciana           594
09 Cataluña                       594
08 Castilla - La Mancha           594
07 Castilla y León                594
06 Cantabria                      594
05 Canarias                       594
04 Balears, Illes                 594
03 Asturias, Principado de        594
02 Aragón                         594
19 Melilla                        594
Name: count, dtype: int64

In [5]:
salario_edad['Grupo de edad'].value_counts()

Grupo de edad
Total              594
De 16 a 24 años    594
De 25 a 34 años    594
De 35 a 44 años    594
De 45 a 54 años    594
55 y más años      594
Name: count, dtype: int64

In [6]:
#PERÍODOS TEMPORALES 
print("\nPERÍODOS TEMPORALES DISPONIBLES:")
for name, df in datasets.items():
    periodo_col = None
    for col in df.columns:
        if 'periodo' in col.lower() or 'año' in col.lower() or 'fecha' in col.lower():
            periodo_col = col
            break

    if periodo_col:
        años_disponibles = df[periodo_col].unique()
        años_recientes = [año for año in años_disponibles if str(año).startswith(('2022', '2023', '2024', '2025'))]
        print(f"{name}: {años_recientes if años_recientes else 'Sin datos recientes'}")
    else:
        print(f"{name}: No se encontró columna de período")



PERÍODOS TEMPORALES DISPONIBLES:
Salarios por Edad: [2023, 2022]
Salarios por Comunidad: [2023, 2022]
IPV Histórico: ['2025T1', '2024T4', '2024T3', '2024T2', '2024T1', '2023T4', '2023T3', '2023T2', '2023T1', '2022T4', '2022T3', '2022T2', '2022T1']
Excel Idealista: [2025, 2024, 2023, 2022]
Superficie Media: Sin datos recientes
Tipos Interés: ['2025M03', '2025M02', '2025M01', '2024M12', '2024M11', '2024M10', '2024M09', '2024M08', '2024M07', '2024M06', '2024M05', '2024M04', '2024M03', '2024M02', '2024M01', '2023M12', '2023M11', '2023M10', '2023M09', '2023M08', '2023M07', '2023M06', '2023M05', '2023M04', '2023M03', '2023M02', '2023M01', '2022M12', '2022M11', '2022M10', '2022M09', '2022M08', '2022M07', '2022M06', '2022M05', '2022M04', '2022M03', '2022M02', '2022M01']


In [7]:
overview(idealista_excel)


Shape: (114, 8)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Comunidades y Ciudades Autonómas  114 non-null    object
 1   Precio€/m2                        114 non-null    object
 2   Variación mensual                 114 non-null    object
 3   Variación trimestral              114 non-null    object
 4   Variación anual                   114 non-null    object
 5   Máximo histórico                  114 non-null    object
 6   Variación máximo                  114 non-null    object
 7   Año                               114 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 7.3+ KB
None

Tipos de datos:
Comunidades y Ciudades Autonómas    object
Precio€/m2                          object
Variación mensual                   object
Variación trimestral                object
Variació

Tras este análisis exploratorio nos damos cuenta que debemos realizar una limpieza de nulos, estandarización de nombres y cambio de tipos de datos. Así como realizar unos cálculos para obtener información de años necesarios. 

# LIMPIEZA DATASETS

In [8]:
# LIMPIAR IDEALISTA
#limpiar precios
idealista_clean = idealista_excel.copy()

In [9]:
def limpiar_precio_m2(precio_texto):
    if pd.isna(precio_texto):
        return None
    precio_limpio = str(precio_texto).replace('€/m2', '').replace('.', '').replace(',', '.').strip()
    try:
        return int(precio_limpio)
    except:
        return None

idealista_clean['Precio m2'] = idealista_clean['Precio€/m2'].apply(limpiar_precio_m2)

print("Precios convertidos:")
print(idealista_clean[['Comunidades y Ciudades Autonómas', 'Precio€/m2', 'Precio m2']].head())

Precios convertidos:
  Comunidades y Ciudades Autonómas  Precio€/m2  Precio m2
0                        Andalucía  2.468 €/m2       2468
1                           Aragón  1.469 €/m2       1469
2                         Asturias  1.538 €/m2       1538
3                         Baleares  4.905 €/m2       4905
4                         Canarias  3.039 €/m2       3039


In [10]:
#ESTANDARIZAR NOMBRES DE CCAA
# Mapeo para unificar nombres entre datasets
mapeo_ccaa = {
    'Andalucía': '01 Andalucía',
    'Aragón': '02 Aragón', 
    'Asturias': '03 Asturias, Principado de',
    'Baleares': '04 Balears, Illes',
    'Canarias': '05 Canarias',
    'Cantabria': '06 Cantabria',
    'Castilla y León': '07 Castilla y León',
    'Castilla-La Mancha': '08 Castilla - La Mancha',
    'Cataluña': '09 Cataluña',
    'Comunitat Valenciana': '10 Comunitat Valenciana',
    'Extremadura': '11 Extremadura',
    'Galicia': '12 Galicia',
    'Madrid Comunidad': '13 Madrid, Comunidad de',
    'Murcia Región': '14 Murcia, Región de',
    'Navarra': '15 Navarra, Comunidad Foral de',
    'Euskadi': '16 País Vasco',
    'La Rioja': '17 Rioja, La',
    'Ceuta': '18 Ceuta',
    'Melilla': '19 Melilla'
}


In [11]:
idealista_clean['Comunidades y Ciudades Autónomas'] = idealista_clean['Comunidades y Ciudades Autonómas'].map(mapeo_ccaa)

In [12]:
columnas_a_mantener_idealista = [
    'Comunidades y Ciudades Autónomas', 
    'Año', 
    'Precio m2'
]

idealista_final = idealista_clean[columnas_a_mantener_idealista].copy()


In [13]:
# ELIMINAR NULOS
superficie_media=drop_nulls(superficie_media)

In [14]:
ipv=drop_nulls(ipv)

In [15]:
from data_cleaning_utils import limpiar_columna_total

In [16]:
#LIMPIEZA DATASET IPV
# Eliminar la columna 'Total Nacional' (solo contiene 'Nacional')
ipv_clean = ipv.drop('Total Nacional', axis=1).copy()

# Renombrar columnas para mayor claridad
ipv_clean = ipv_clean.rename(columns={
    'General, vivienda nueva y de segunda mano': 'Tipo Vivienda',
    'Índices y tasas': 'Tipo Indicador'
})

In [17]:
#LIMPIEZA DATASET SUPERFICIE MEDIA
#ESTANDARIZAR NOMBRES DE CCAA
# Mapeo para unificar nombres entre datasets
mapeo_ccaa_2 = {
    'Andalucía': '01 Andalucía',
    'Aragón': '02 Aragón', 
    'Asturias, Principado de': '03 Asturias, Principado de',
    'Balears, Illes': '04 Balears, Illes',
    'Canarias': '05 Canarias',
    'Cantabria': '06 Cantabria',
    'Castilla y León': '07 Castilla y León',
    'Castilla - La Mancha': '08 Castilla - La Mancha',
    'Cataluña': '09 Cataluña',
    'Comunitat Valenciana': '10 Comunitat Valenciana',
    'Extremadura': '11 Extremadura',
    'Galicia': '12 Galicia',
    'Madrid, Comunidad de': '13 Madrid, Comunidad de',
    'Murcia, Región de': '14 Murcia, Región de',
    'Navarra, Comunidad Foral de': '15 Navarra, Comunidad Foral de',
    'País Vasco': '16 País Vasco',
    'Rioja, La': '17 Rioja, La',
    'Ceuta': '18 Ceuta',
    'Melilla': '19 Melilla'
}

In [18]:
superficie_media['Comunidades y Ciudades Autónomas'] = superficie_media['Comunidades y Ciudades Autónomas'].map(mapeo_ccaa_2)

/var/folders/4q/413qpbj17ks9bgdzl52cpx9m0000gn/T/ipykernel_2027/2539730385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  superficie_media['Comunidades y Ciudades Autónomas'] = superficie_media['Comunidades y Ciudades Autónomas'].map(mapeo_ccaa_2)


In [19]:
superficie_clean = superficie_media.drop('Total Nacional', axis=1).copy()

# Renombrar columnas
superficie_clean = superficie_clean.rename(columns={
    'Superficie útil de la vivienda': 'Superficie Util',
    'Tamaño del hogar': 'Tamaño Hogar',
    'periodo': 'Año'
})

In [20]:
#LIMPIEZA COLUMNA TOTAL EN TODOS LOS CSV (menos idealista)
ipv_clean= limpiar_columna_total(ipv_clean)
superficie_clean= limpiar_columna_total(superficie_clean)
salario_comunidad = limpiar_columna_total(salario_comunidad)
salario_edad = limpiar_columna_total(salario_edad)
interes_fijo = limpiar_columna_total(interes_fijo)

In [21]:
salario_comunidad.rename(columns={'Comunidades y Ciudades Autonómas': 'Comunidades y Ciudades Autónomas'}, inplace=True)

In [22]:
#estandarizar al completo la variables de la columna de ccaa
salario_comunidad = limpiar_comillas_ccaa(salario_comunidad)
ipv_clean = limpiar_comillas_ccaa(ipv_clean)
idealista_final = limpiar_comillas_ccaa(idealista_final)

In [ ]:
# columna de año
salario_comunidad.rename(columns={"Periodo": "Año"}, inplace=True)
salario_edad.rename(columns={"Periodo": "Año"}, inplace=True)

In [23]:
# Data time en Periodo de IPV e Interés Fijo
ipv_clean = convertir_periodo_con_detalles(ipv_clean)
interes_fijo = convertir_periodo_con_detalles(interes_fijo)


In [24]:
#todos a int64
ipv_clean = convertir_int32_a_int64(ipv_clean)
interes_fijo = convertir_int32_a_int64(interes_fijo)

EXPORTAMOS LOS DOS CSVS LIMPIOS 
import os
os.makedirs('data/clean', exist_ok=True)
superficie_clean.to_csv('data/clean/superficie_clean.csv', index=False)
ipv_clean.to_csv('data/clean/ipv_clean.csv', index=False)
salario_comunidad.to_csv('data/clean/salario_ccaa_clean.csv', index=False)
salario_edad.to_csv('data/clean/salario_edad_clean.csv', index=False)
idealista_final.to_csv('data/clean/idealista_clean.csv', index=False)
interes_fijo.to_csv('data/clean/interes_clean.csv', index=False)

idealista_final, interes_fijo, slario_comunidad, salario_edad y el resto "_clean" son los datasets limpios para trabajar en ellos 